本节主要讨论如何处理文本数据

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image001.png)

理解词嵌入(word embedding)

模型无法直接处理文本数据，模型只能接受数值型数据，因此第一步是如何将文本数据转化为数值型数据，最后一步是如何将数值型数据转为文本数据，就是一个编码和解码的过程

将数据转为向量形式的方法就是embedding

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image003.png)

常用的embedding方法有one-hot,word2vec,byte pair encoding </br>

one-hot主要用在分类之中，是一种简单直观的方法，适用于类别特征的编码，特别是在类别数量不是非常大时。然而，当类别数量很多或者需要捕捉特征之间复杂关系的场景下。

原始数据：</br>
样本1：狗</br>
样本2：猫</br>
样本3：鸟</br>
样本4：狗</br>
One-hot编码步骤：</br>
定义类别集合：首先确定所有可能的类别。在这个例子中，我们有三种动物：狗、猫、鸟。</br>
为每个类别创建一个维度：每个类别都会有一个对应的维度，值为0或1。在这个例子中，我们将有三个维度，分别对应三种动物。</br>
编码每个样本：将每个样本转换为one-hot向量。</br>
样本1（狗）的one-hot编码：[1, 0, 0]</br>
样本2（猫）的one-hot编码：[0, 1, 0]</br>
样本3（鸟）的one-hot编码：[0, 0, 1]</br>
样本4（狗）的one-hot编码：[1, 0, 0]

Word2Vec是一种用于生成词嵌入（word embeddings）的模型，由Tomas Mikolov和他的团队在2013年提出。它通过训练神经网络语言模型来将词汇映射到向量空间，使得语义上相似的词在该空间中距离更近.
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image005.png)

我们也可以直接使用Word2Vec来编码文本数据，但业内大模型一般都是使用自己的方式来训练embedding，并且作为输入层的一部分。我们会在本章实现embedding层。

gpt2的embedding size是768维，gpt3是12288维。

接下来我们开始学习如何训练自己的embedding,包括将文本数据分割成word，然后将word转化为token，最终将token转为embedding vector.

token是词和字符(比如标点符号)，在大模型使用中，是按照token使用量来计费的。

#### Tokenizing text
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image007.png)

In [56]:
# 这里我们使用一本版权开放的小说The Verdict.
# 这里先使用python把文本数据读入进来
with open('the-verdict.txt',mode="r",encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:",len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [55]:
# 分割文本数据
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)',text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [58]:
#将,.进行独立出来
result = re.split(r'([,.]|\s)',text)
#result = re.split(r'([,.\s])',text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [59]:
# 将空白字符去掉
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


是否去除空格，取决于数据，比如python这类的代码，对于空格是敏感的，这样的数据是不能去除空格的

In [60]:
# 更完善的做法，适应复杂的特殊字符
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!\']|--|\s)',text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image009.png)

In [61]:
#对verdict进行整个tokenized
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [62]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


#### 3.Coverting tokens into token IDs

在token转为embedding vector前，还需要一个中间步，就是将token转为token ID，通过token ID输入训练模型

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image011.png)

In [63]:
# 创建词汇表vocabulary
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [64]:
vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item) 
    if i>50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


在大模型输出层，需要将token ID转为对应的token，这个过程相当于decode，因此我们可以编写一个class，里面有encode和decode方法

In [66]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        ids = [self.str_to_int[item] for item in preprocessed if item.strip()]
        return ids
    
    def decode(self,ids):
        text = " ".join([self.int_to_str[id] for id in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])',r'\1',text)
        return text

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image015.png)

In [67]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [68]:
#再用decode解码回来text 
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [69]:
# 使用不在小说里的数据看看tokenization的结果
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

#### 4.Adding special context tokens
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image017.png)

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image019.png)

In [70]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [71]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [72]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
        ids = [self.str_to_int[item] for item in preprocessed ]
        return ids
    
    def decode(self,ids):
        text = " ".join([self.int_to_str[id] for id in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])',r'\1',text)
        return text

In [73]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [74]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [75]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


上面是简单的tokenization方法，不过GPT模型中没有使用<|unk|>来表示不认识的token，而是使用了一种叫byte pair encoding的方法。

#### 5.Byte pair encoding（BPE）
BPE实现方式比较复杂，openai开源了一个BPE库（https://github.com/openai/tiktoken），底层使用rust，非常高效，可以拿来直接使用，在后面的分享中，找机会和大家详细讲解BPE的实现过程

In [ ]:
# %pip install tiktoken

In [76]:
#检查tiktoken的版本号
from importlib.metadata import version
import tiktoken

print("tiktoken version:",version("tiktoken"))

tiktoken version: 0.6.0


In [77]:
#初始化一个BPE tokenizer实例
tokenizer = tiktoken.get_encoding("gpt2")

In [78]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [79]:
tokenizer1 = SimpleTokenizerV2(vocab)
integers1 = tokenizer1.encode(text)
text_decode = tokenizer1.decode(integers1)
print(len(integers1))
print(len(integers))
print(text_decode)

15
20
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of <|unk|>.


In [ ]:
# integers = [34680]

In [83]:
text = "some unknownsdfsdfsello"
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[11246, 6439, 82, 7568, 82, 7568, 7255, 78]


In [88]:
token_ids=[6439]
print(len(tokenizer.decode(token_ids)))

8


In [89]:
text = "unknown"
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[34680]


In [82]:
strings = tokenizer.decode(integers)
print(len(strings))

17


在这个过程中，可以发现BPE并没有去掉空格

#### 6.Data sampling with a sliding window

大模型是根据输入内容来预测下一个token，预训练阶段需要准备输入-目标对，如下图：
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image023.png)

在这里我们使用了一种称为滑动窗口的方法来实现输入-目标对(input-target pair)

In [90]:
# 第一步先使用上面的tokenizer类将verdict文本token化
with open(file="the-verdict.txt",encoding="utf-8",mode="r") as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [91]:
enc_sample = enc_text[50:]

In [92]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:     {y}")

x: [290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [93]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context,"---->",desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [94]:
# 看下具体文本形式的
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context),"---->",tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


我们还需要一个DataLoader,将数据通过DataLoader进行加载
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image025.png)

In [ ]:
#根据上一讲，咱们先实现DataSet
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(txt)
        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

In [ ]:
# 创建Dataloader
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,
                         drop_last=True,num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataloader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last=drop_last,
                            num_workers=num_workers)
    return dataloader

In [95]:
#测试下dataloader效果
with open(file="the-verdict.txt",encoding="utf-8",mode="r") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text,batch_size=1,max_length=4,stride=1,shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [96]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


stride相当于一个滑动窗口，决定着训练数据重合程度，如果重合太多，训练可能过拟合
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image027.png)

In [97]:
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=4,stride=4,shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


#### 7.creating token embeddings
最后一步是如何将token id转为embeddings

![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image029.png)

In [98]:
# 假设词典表只有6个token
input_ids = torch.tensor([2,3,5,1])

In [99]:
#embeddings vector有3维
vocab_size = 6
output_dim = 3

In [100]:
#建立一个embedding神经网络层
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size,output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


上面embedding层参数会在训练过程中变化

In [101]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [102]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image031.png)

#### 8.Encoding word positions
一句话里同样的字在不同的位置表达的含义是不一样的，transformer模型使用的自注意力机制一个缺点是没有关注到位置信息。而上面讲的embedding方法对不同位置的token得出的值是一样的。
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image033.png)

没有位置信息也有好处就是结果可以重现，但加入信息可以让对数据理解更全面和充分
![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image035.png)

位置embedding信息和token的embedding有一样的维度，两者相加就是输入embedding，上图展示的是绝对位置信息

还有一种位置信息是相对位置嵌入，它是一种使模型能够理解和处理不同长度序列的技术，它通过让模型学习标记之间的相对距离而不是它们的绝对位置，提高了模型的泛化能力和适用性。这对于构建能够处理多样化序列长度的深度学习模型非常重要。

绝对位置潜入和相对位置潜入各有千秋，根据不同的数据和任务，可以选择合适的方法。

在transformer论文中，位置信息是固定的。GPT大模型使用了绝对位置嵌入，但会在训练过程中进行优化。


In [108]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [109]:
with open(file="the-verdict.txt",encoding="utf-8",mode="r") as f:
    raw_text = f.read()
max_length = 4
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n",inputs)
print("\nInputs shape:\n",inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [110]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [111]:
#绝对位置信息嵌入
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [112]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


![](https://drek4537l1klr.cloudfront.net/raschka/v-8/Figures/ch02__image037.png)